## Upload artifact registry
Estos códigos se podrían correr en la consola SDK de google, así como correrlos en un notebook.
Para un data scientist poco especializado en prácticas de devops, utilizar un notebook es más inuitivo de utilizar

##### Pasos previos
- Pararse en la carpeta root de la app (crear este notebook en el root)
- Tener creados scripts a contenizar y subirse a cloud run


revisar este punto
- Crear Dockerfile en el root (un ejemplo de Dockerfile se puede encontrar en: https://firebase.google.com/docs/hosting/cloud-run?hl=es-419#python)

##### Estructura Notebook Deploy app web
Existen dos métodos para hacer el deploy, almacenar la imagen docker en artifact registry (forma actual) y almacenar la imagen en container registry (forma antigua y cada vez más deprecated). En este Notebook por ser el primer y el de ejemplo se va a hacer el deploy de ambas formas primero en artifact registry y posteriormente en container registry

## INICIALIZAR: SET PROYECT GCP

**Para correr comando de consola en un jupyter notebook y que además se le puedan pasar variables del notebook al comando se debe utilizar el signo peso ($) y no utilizar el comando de asignación (=)**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables """
env_var_project_gcp = os.environ.get("PROJECT_GCP", "")

# SET SERVICE ACCOUNT GCP AND PROJECT
PROJECT_ID = env_var_project_gcp

In [1]:
REGION = 'us-east1'
! gcloud config set project $PROJECT_ID

Updated property [core/project].


## ------- ARTIFACT REGISTRY -------

Artifact registry es el reemplazo de container registry y el recomendado por google. La única diferencia es que se guarda la imagen en este nuevo servicio y se necesita correr otros comando


Además, cada vez que se sube una imagen nueva a artifact registry (volver a ejecutar el comando correspondiente de gcloud), esta recibe la etiqueta latest y es la que se utiliza para crear/actualizar la cloud run creada

Subir la imagen a artifact Registry requiere más pasos que subirla a container registry

Documentación integración cloud build con Artifact Registry: https://cloud.google.com/artifact-registry/docs/configure-cloud-build?hl=es-419

### Paso 0: Parámetros

In [2]:
# PARÁMETROS 

# generales gcp
REGION = 'us-east1'

# nombre del repo
NAME_REPO = 'repo-app-web-6-hello-world-refresh-event-observer-bq-v2'
FORMAT_REPO = 'docker'
DESCRIPTION_REPO = "repo web app 6 hello world refresh event observer bq v2 socketio"

# nombre de la imagen
NAME_IMAGE = '6-hello-world-refresh-event-observer-bq-v2'

# nombre del cloud run que va a alojar la app web
NAME_CLOUD_RUN = 'app-web-6-hello-world-refresh-event-observer-bq-v2'

### Paso 1. Crear repositorio en artifact registry (si es que este no existe)
A diferencia de container registry que era automática, en artifact registry hay que crearlo.

En container registry solo se creaba una imagen y se almacenaban diferentes versiones de la imagen, por el contrario en artifact registry se crea un repo el cual puede tener múltiples imágenes y cada una tener diferentes versiones


Documentación: crear repo en artifact registry: https://cloud.google.com/artifact-registry/docs/repositories/create-repos#gcloud

In [3]:
# crear repo
! gcloud artifacts repositories create $NAME_REPO \
--repository-format $FORMAT_REPO \
--location $REGION \
--description "$DESCRIPTION_REPO" \
--async

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


### Paso 2: Configurar una compilación de Docker

Es necesario crear un **yaml** con la configuración para compilar la imagen docker en Artifact Registry.

Tiene la siguiente forma

<code>
steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}', '.' ]
images:
- '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}'
<code>
    

**Este un ARCHIVO GENÉRICO que se puede reciclar porque está parametrizado para funcionar con cualquier repo docker en artifact registry** 

--> Corriendo la siguiente línea de código se crea un archivo yaml con la configuración deseada. 

Documentación: https://stackabuse.com/reading-and-writing-yaml-to-a-file-in-python/

In [4]:
import yaml

In [5]:
# crear diccionario python con el contenido del yaml cloudbuild genérico

dict_python_yaml_cloudbuild = {'steps': [{'name': 'gcr.io/cloud-builders/docker',
   'args': ['build', '-t', '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}', '.']}],
 'images': ['${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}']}

In [6]:
# guardar diccionario en formato yaml

with open(r'cloudbuild.yaml', 'w') as file:
    documents = yaml.dump(dict_python_yaml_cloudbuild, file)

### Paso 3. Crear Dockerfile

Igual que el paso anterior, se necesita crear en este caso, el Dockerfile para poder subir la imagen al Artifact Registry. Por lo general, el dockerfile, se crea de forma manual.

En este ejemplo, se define el contenido del dockerfile dentro de un string lo que permite que en un futuro se pueda cambiar vía código por ejemplo:
- versión de python
- Listado de archivos que van a formar parte de la imagen
- Librerías que necesita la imagen para funcionar

Por lo general, la mayoría de las veces, el Dockerfile no necesita ser modificado salvo que se desee cambiar la versión de python o que se requiera agregar nuevas librerías en el desarrollo. Por lo que corriendo el sgte código se obtiene el Dockerfile de la app web

https://prnt.sc/uwBOFChK8QU8

In [7]:
# crear string que representa al dockerfile

string_dockerfile = '''FROM python:3.7.5-slim-buster
ENV PYTHONUNBUFFERED True

# Copy local code to the container image.
ENV APP_HOME /app
WORKDIR $APP_HOME
COPY . ./

# Install production dependencies.
RUN pip install Flask gunicorn pandas numpy plotly google-cloud-bigquery db-dtypes Flask-Caching python-socketio==5.3.0

CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 --timeout 0 app:app
'''

In [8]:
# guardar dockerfile
with open('Dockerfile', 'w') as file:
    file.write(string_dockerfile)

### Paso 4: Contenerizar (imagen docker) códigos app web utlizando cloud build y subirlas a artifact registry
- En este paso se crea una imagen docker con los códigos necesarios para la app web y posterior se procede a subir dicha imagen a Artifact Registry (utilizando como base el archivo "cloudbuild.yaml" que llama al "Dockerfile", creados en los pasos anteriores)

In [9]:
# OJO: Los nombres de las variables en el comando gcloud corresponden a las variables definidas en el archivo de configuración 
# yaml

# OJO2: para pasar el nombre de las variables (al igual que siempre) se debe de utilizar el signo dolar "$" pero debe
# de estar entre comillas (para que se entienda que es la variable a reemplazar en el yaml de configuración)

# OJO3: debe ser comillas dobles y sin espacios para no tener problemas

! gcloud builds submit \
    --config=cloudbuild.yaml \
    --substitutions=_LOCATION="$REGION",_REPOSITORY="$NAME_REPO",_IMAGE="$NAME_IMAGE" .

----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "de7d7193-7cd4-4247-bd9d-e6cac6f1129c"

FETCHSOURCE
Fetching storage object: gs://cmpc-innovation-cd4ml-test_cloudbuild/source/1678151568.358685-71898a5e72a4431f99aedaa42c561b6a.tgz#1678151569242042
Copying gs://cmpc-innovation-cd4ml-test_cloudbuild/source/1678151568.358685-71898a5e72a4431f99aedaa42c561b6a.tgz#1678151569242042...
/ [0 files][    0.0 B/ 30.5 KiB]                                                
/ [1 files][ 30.5 KiB/ 30.5 KiB]                                                

Operation completed over 1 objects/30.5 KiB.
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker
Sending build context to Docker daemon    107kB


Step 1/7 : FROM python:3.7.5-slim-buster
3.7.5-slim-buster: Pulling from library/python
000eee12ec04: Pulling fs layer
ddc2d83f8229: Pulling fs layer
735b0bee82a3: Pulling fs layer
8c69dcedfc84: Pulling fs layer
495e1cccc7f9: Pulling fs layer
8c

Creating temporary tarball archive of 12 file(s) totalling 91.7 KiB before compression.
Uploading tarball of [.] to [gs://cmpc-innovation-cd4ml-test_cloudbuild/source/1678151568.358685-71898a5e72a4431f99aedaa42c561b6a.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/cmpc-innovation-cd4ml-test/locations/global/builds/de7d7193-7cd4-4247-bd9d-e6cac6f1129c].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/de7d7193-7cd4-4247-bd9d-e6cac6f1129c?project=724348686027].


### Paso 5:  Deploy de la imagen del contenedor de artifact registry en cloud run
Se debe de correr el comando gloud run deploy (igual que en el container registry) con la única diferencia en que cambia la ubicación de la imagen la cual es:

 {LOCATION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{IMAGE}/
 
 

**IMPORTANTE: POR PROBLEMAS DE PERMISOS, EL CLOUD RUN ESTÁ CONFIGURADO PARA QUE CUALQUIERA CON EL LINK PUEDA ACCEDER**

#### Importante: está hardcodeado el timeout y la memoria

In [10]:
! gcloud run deploy $NAME_CLOUD_RUN \
    --image $REGION-docker.pkg.dev/$PROJECT_ID/$NAME_REPO/$NAME_IMAGE \
    --region $REGION \
    --timeout=1000 \
    --memory 512Mi \
    --set-env-vars=PROJECT_GCP=$env_var_project_gcp \
    --allow-unauthenticated

Deploying container to Cloud Run service [app-web-6-hello-world-refresh-event-observer-bq-v2] in project [cmpc-innovation-cd4ml-test] region [us-east1]
Deploying...
Setting IAM Policy...........................done
Creating Revision..............................................................................................................................................................................................................................................................................done
Routing traffic.......................................................done
Done.
Service [app-web-6-hello-world-refresh-event-observer-bq-v2] revision [app-web-6-hello-world-refresh-event-observer-bq-v2-00003-has] has been deployed and is serving 100 percent of traffic.
Service URL: https://app-web-6-hello-world-refresh-event-observer-bq-v-z33tzzez7a-ue.a.run.app


In [11]:
# memory gigas

###--memory 1Gi \